<h1>Data Processing</h1>

<h3>Importpath</h3>

Add Projectfolder to Importpath

In [1]:
import sys
import os
cwd = os.path.abspath(".")
sys.path.insert(0, cwd.replace("Notebook", "Code"))

sys.path

['/home/avatar/Dokumente/Projekte/Python/EconomicCalendarImpactAnalysis/Code',
 '/home/avatar/Dokumente/Projekte/Python/EconomicCalendarImpactAnalysis/Notebook',
 '/home/avatar/.pyenv/versions/3.10.13/lib/python310.zip',
 '/home/avatar/.pyenv/versions/3.10.13/lib/python3.10',
 '/home/avatar/.pyenv/versions/3.10.13/lib/python3.10/lib-dynload',
 '',
 '/home/avatar/.local/lib/python3.10/site-packages',
 '/home/avatar/.pyenv/versions/3.10.13/lib/python3.10/site-packages']

<h3>Imports</h3>

In [2]:
import pandas as pd

import fmp
import influx as db

<h3>Data Processing</h3>

In [3]:
start_date = pd.Timestamp("2024-01-01")
stop_date = pd.Timestamp("2024-01-07")

FMP = fmp.FinancialModelingPrep()
FMP.economic_calendar_pipeline(start_date, stop_date)

[{'date': '2024-02-10 00:00:00',
  'country': 'ID',
  'event': 'Chinese New Year',
  'currency': 'IDR',
  'previous': None,
  'estimate': None,
  'actual': None,
  'change': 0,
  'impact': 'Low',
  'changePercentage': 0},
 {'date': '2024-02-10 00:00:00',
  'country': 'CN',
  'event': 'Chinese New Year',
  'currency': 'CNY',
  'previous': None,
  'estimate': None,
  'actual': None,
  'change': 0,
  'impact': 'Low',
  'changePercentage': 0},
 {'date': '2024-02-10 00:00:00',
  'country': 'HK',
  'event': 'Chinese New Year',
  'currency': 'HKD',
  'previous': None,
  'estimate': None,
  'actual': None,
  'change': 0,
  'impact': 'Low',
  'changePercentage': 0},
 {'date': '2024-02-10 00:00:00',
  'country': 'SG',
  'event': 'Chinese New Year',
  'currency': 'SGD',
  'previous': None,
  'estimate': None,
  'actual': None,
  'change': 0,
  'impact': 'Low',
  'changePercentage': 0},
 {'date': '2024-02-10 00:00:00',
  'country': 'PH',
  'event': 'Chinese New Year',
  'currency': 'PHP',
  'previ